# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../city_weather_csv')
weather_df

,City,Country,Lat,Lon,Max Temp,Humidity (%),Cloudiness (%),Wind Speed (mph),Date
0,clyde river,CA,70.47,-68.59,28.40,92,90,3.36,1590259298
1,bredasdorp,ZA,-34.53,20.04,59.00,62,0,5.82,1590259299
2,jamestown,US,42.10,-79.24,66.99,88,90,6.93,1590259170
3,sisimiut,GL,66.94,-53.67,40.50,71,4,18.28,1590259301
4,salalah,OM,17.02,54.09,86.00,89,39,4.70,1590259119
...,...,...,...,...,...,...,...,...,...
550,soure,BR,-0.72,-48.52,82.90,81,23,12.35,1590259891
551,herne bay,GB,51.37,1.13,62.01,59,0,5.01,1590259892
552,ancud,CL,-41.87,-73.82,59.00,100,90,17.22,1590259893
553,maragogi,BR,-9.01,-35.22,80.67,85,18,10.18,1590259894


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Use gmaps tp create heatmap 
gmaps.configure(api_key=g_key)

locations = weather_df[['Lat', 'Lon']]
weights = weather_df['Humidity (%)']

fig1 = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights)
fig1.add_layer(heatmap_layer)
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 8

fig1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
weather_df_clean = weather_df.dropna()

#filter to cities with ideal weather conditions
weather_df_ideal = weather_df_clean.loc[(weather_df["Max Temp"] < 85) & 
                                     (weather_df["Max Temp"] > 65) &    
                                     (weather_df["Humidity (%)"] < 25) &
                                     (weather_df["Cloudiness (%)"] < 10),:]

weather_df_ideal

,City,Country,Lat,Lon,Max Temp,Humidity (%),Cloudiness (%),Wind Speed (mph),Date
27,saint george,US,37.10,-113.58,69.80,22,1,9.17,1590259327
41,san juan,AR,-31.54,-68.54,66.06,24,0,6.42,1590259342
103,chino valley,US,34.76,-112.45,75.00,19,1,10.29,1590259408
125,beyneu,KZ,45.32,55.20,83.79,22,0,21.99,1590259431
208,herat,AF,34.50,62.00,75.20,23,0,6.93,1590259521
240,sabha,LY,27.04,14.43,81.72,23,0,23.33,1590259556
275,laguna,US,38.42,-121.42,77.00,20,1,12.75,1590259593
426,raton,US,36.90,-104.44,73.40,8,1,16.11,1590259756
458,ridgecrest,US,35.62,-117.67,75.20,11,1,5.82,1590259792
461,garmsar,IR,35.22,52.34,84.51,18,0,6.26,1590259795


### Hotel Map
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotels = []

target_radius = 5000
target_type = "lodging"

#loop through cities to show lat and lon
for index, row in weather_df_ideal.iterrows():
    lat = row["Lat"]
    lon = row['Lon']

    # set up a parameters dictionary
    params = {
        "location": (f"{lat}, {lon}"),
        "radius": target_radius,
        "type": target_type,
        "key": g_key}

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    response.url
    
    places_data = response.json()

    #skip if no hotels found
    if len(places_data['results']) == 0:
        print(f"skip city")
        hotels.append("none") 
    else:
        print(places_data['results'][0]['name']) 
        hotels.append(places_data['results'][0]['name']) 
       

Best Western Plus Abbey Inn
Hotel Provincial
Primrose Inn & Suites
NUR
skip city
محطة ام الارانب -سبها مواصلات
Holiday Inn Express & Suites Elk Grove Central - Hwy 99
Holiday Inn Express & Suites Raton
SpringHill Suites by Marriott Ridgecrest
Peyvand Hotel
WASIM ASGAR KHAN


In [6]:
weather_df_ideal['Hotel Name'] = hotels
weather_df_ideal

C:\Users\katie\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Country,Lat,Lon,Max Temp,Humidity (%),Cloudiness (%),Wind Speed (mph),Date,Hotel Name
27,saint george,US,37.10,-113.58,69.80,22,1,9.17,1590259327,Best Western Plus Abbey Inn
41,san juan,AR,-31.54,-68.54,66.06,24,0,6.42,1590259342,Hotel Provincial
103,chino valley,US,34.76,-112.45,75.00,19,1,10.29,1590259408,Primrose Inn & Suites
125,beyneu,KZ,45.32,55.20,83.79,22,0,21.99,1590259431,NUR
208,herat,AF,34.50,62.00,75.20,23,0,6.93,1590259521,none
240,sabha,LY,27.04,14.43,81.72,23,0,23.33,1590259556,محطة ام الارانب -سبها مواصلات
275,laguna,US,38.42,-121.42,77.00,20,1,12.75,1590259593,Holiday Inn Express & Suites Elk Grove Central...
426,raton,US,36.90,-104.44,73.40,8,1,16.11,1590259756,Holiday Inn Express & Suites Raton
458,ridgecrest,US,35.62,-117.67,75.20,11,1,5.82,1590259792,SpringHill Suites by Marriott Ridgecrest
461,garmsar,IR,35.22,52.34,84.51,18,0,6.26,1590259795,Peyvand Hotel


In [7]:
# template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in weather_df_ideal.iterrows()]
locations = weather_df_ideal[["Lat", "Lon"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig1.add_layer(marker_layer)

fig1

Figure(layout=FigureLayout(height='420px'))